#### Modelo baseado em Ilhas
- Topologia Anel

<img src="https://servicewordpressigti.azurewebsites.net/wp-content/uploads/2022/09/topologia-de-rede-anel-1024x837.jpg" width="200"
  height="200" align="left"/>


#### Técnicas de implementação
* Técnica de Migração: migRing (Framework DEAP)

#### Programação concorrente
* SCOOP: (https://scoop.readthedocs.io/en/0.7/)

In [1]:
!pip install deap
!pip install scoop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scoop: filename=scoop-0.7.2.0-py3-none-any.whl size=78113 sha256=3fed45021b7a176e10a2004211bc7d8104517a75b055dcd9007a089498c5bf91
  Stored in directory: /root/.cache/pip/wheels/26/c6/da/088e4bffcfbc33fa40644ca636267bc801c8fd9eef973483db
Successfully built scoop


In [2]:
import array
import random
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# Bibliteca execução paralelismo
from scoop import futures

In [3]:
def fitnessFunction(chromosome):
    return sum(chromosome),

In [17]:
# Cria o tipo de função fitness e indivíduo
creator.create("Maximization", base.Fitness, weights=(1.0,))
creator.create("Genes", list,fitness=creator.Maximization)

toolbox = base.Toolbox()
# Registra os nomes e os tipos de individuo, fiteness e população
toolbox.register("Atributo", random.randint, 0, 1)
toolbox.register("Cromossomo", tools.initRepeat, creator.Genes, toolbox.Atributo, n=300 )
toolbox.register("Populacao", tools.initRepeat, list, toolbox.Cromossomo)

# Registra os operadores. Deve-se manter os nomes evaluate, mate, mutate e select
toolbox.register("evaluate", fitnessFunction)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.01)
toolbox.register("select", tools.selTournament, tournsize = 5)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Maximization' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Genes' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [18]:
# Registra a execução em paralismo
toolbox.register("map", futures.map)

In [25]:
# Parâmetros do Algoritmo
prob_cx = 0.8
prob_mt = 0.1
nger = 400
freq = 10
tamPop = 30
nIlhas = 5

In [26]:
# Faz a execução do AG em Ilhas
islands = [toolbox.Populacao(n=tamPop) for _ in range(nIlhas)]
toolbox.register("algorithm", algorithms.eaSimple, toolbox=toolbox,cxpb=prob_cx,
                 mutpb = prob_mt, ngen=freq, verbose=False)

for _ in range(0, nger, freq):
  results = toolbox.map(toolbox.algorithm, islands)
  islands = [pop for pop, logbook in results]
  tools.migRing(islands, 5, tools.selBest)

In [27]:
# Imprime o fitness máximo e médio por Ilhas
for pop in islands:
  # Armazena o Fitness final de cada Ilha
  finalFitness = []
  for individual in pop:
    finalFitness.append(individual.fitness.values)
  print(f'Max Fitness: {max(finalFitness)} - Avg Fitness: {numpy.mean(finalFitness)}')

Max Fitness: (300.0,) - Avg Fitness: 299.3
Max Fitness: (300.0,) - Avg Fitness: 299.5
Max Fitness: (300.0,) - Avg Fitness: 299.9
Max Fitness: (300.0,) - Avg Fitness: 299.6333333333333
Max Fitness: (300.0,) - Avg Fitness: 299.7
